In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['Data']


## This is project we will try to predict stock market dataset

In [2]:
##lets import some more dependencies
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import plotly 
import datetime

In [5]:
## now lets check the dataset
stock_file_names = os.listdir("../input/Data/Stocks")

In [6]:
stock_file_names


['lake.us.txt',
 'avt.us.txt',
 'jps.us.txt',
 'form.us.txt',
 'un.us.txt',
 'kcapl.us.txt',
 'fbiz.us.txt',
 'amh_c.us.txt',
 'sce_k.us.txt',
 'avdl.us.txt',
 'eeq.us.txt',
 'dox.us.txt',
 'dlng.us.txt',
 'ups.us.txt',
 'avgo.us.txt',
 'wmc.us.txt',
 'qdel.us.txt',
 'tgna.us.txt',
 'puk.us.txt',
 'bcacw.us.txt',
 'vbiv.us.txt',
 'wvvip.us.txt',
 'hmta.us.txt',
 'inf.us.txt',
 'mnkd.us.txt',
 'cli.us.txt',
 'alo.us.txt',
 'rail.us.txt',
 'fomx.us.txt',
 'pgp.us.txt',
 'rtix.us.txt',
 'vbnd.us.txt',
 'town.us.txt',
 'hbb.us.txt',
 'yext.us.txt',
 'akg.us.txt',
 'vsmv.us.txt',
 'feng.us.txt',
 'hbk.us.txt',
 'grnb.us.txt',
 'asc.us.txt',
 'tlra.us.txt',
 'cae.us.txt',
 'gis.us.txt',
 'aamc.us.txt',
 'ottr.us.txt',
 'lvnta.us.txt',
 'vtc.us.txt',
 'cltl.us.txt',
 'ocn.us.txt',
 'mmdmr.us.txt',
 'regi.us.txt',
 'all_a.us.txt',
 'gnk.us.txt',
 'oran.us.txt',
 'blue.us.txt',
 'psa_f.us.txt',
 'appf.us.txt',
 'oeuh.us.txt',
 'f.us.txt',
 'tvpt.us.txt',
 'nghc.us.txt',
 'enva.us.txt',
 'hmy.us

In [7]:
stockSymbol=[]
col_names=['Date','Open','High','Low','Close','Volume','c']
dataList=list()
for f in stock_file_names:
    symbol=[str.upper(f.split('.')[0])]
    fileName='../input/Data/Stocks/'+f
    fp=open(fileName,'r')
    if len(fp.read())>0:
        data=np.array(pd.read_csv(fileName,header=None))
        for d in data[1:]:
            dataList.append(d)
            stockSymbol.append(symbol)
        
    fp.close()
print('DONE!!')

DONE!!


In [8]:
## because list has no attribute called shape
## so converted into numpy array to find out 
## the shape
np.shape(dataList),np.shape(stockSymbol)

((14887665, 7), (14887665, 1))

In [9]:
## lets see the first items of both the lists
print('the datalist item is : ',dataList[0])
print('the stock symbol item is : ',stockSymbol[0])


the datalist item is :  ['2005-02-25' '16.69' '16.86' '16.69' '16.86' '25195' '0']
the stock symbol item is :  ['LAKE']


In [10]:
## lets create the dataframe
df1 = pd.DataFrame(dataList,columns=col_names)
df2 = pd.DataFrame(stockSymbol,columns = ['SYMBOL'])

In [11]:
df2.head(5)

,SYMBOL
0,LAKE
1,LAKE
2,LAKE
3,LAKE
4,LAKE


In [12]:
all_stock_symbols = df2.SYMBOL.unique()

In [13]:
len(all_stock_symbols)


7163

In [14]:
df1.head(3)

,Date,Open,High,Low,Close,Volume,c
0,2005-02-25,16.69,16.86,16.69,16.86,25195,0
1,2005-02-28,16.86,17.36,16.67,17.35,34846,0
2,2005-03-01,17.44,17.67,17.25,17.49,12703,0


In [15]:
df = pd.concat([df1,df2],axis=1)

In [ ]:
df.head(3)

In [ ]:
df.head()

In [16]:
## lets try as in the example , with close dataframe
all_close_df = df[['Date','Close','SYMBOL']]

In [ ]:
all_close_df.head()

In [18]:
def getCloseDataFromSymbols(df,symbol_list):
    return df[df['SYMBOL'].isin(symbol_list)][['Date','Close']]

resultDF=getCloseDataFromSymbols(all_close_df,['AMZN','FB','GOOGL','NFLX'])
len(resultDF)

13068

In [19]:
def getPastSequenceData(df,window):
    #df=np.array(df)
    X=[]
    y=[]
    for i in range(1,len(df)-window,window):
        #print(df[i-1:i+window-1],df[i+window-1])
        date=df['Date'].iloc[i]
        date=date.replace('-','')
        #print(date)
        date=np.array(date)
        vals=np.array(df['Close'].iloc[i-1:i+window-1].values)
        temp=np.hstack([date,vals])
        X.append(temp)
        y.append(df['Close'].iloc[i+window-1])
    return X,y

In [20]:
X,y=getPastSequenceData(resultDF,7)
X=np.array(X)
print(X.shape)
X=X.reshape(X.shape[0],X.shape[1],1)
y=np.array(y)
np.shape(X),np.shape(y)

(1866, 8)


((1866, 8, 1), (1866,))

In [21]:
from sklearn.model_selection import TimeSeriesSplit


In [22]:
tsSplit=TimeSeriesSplit(n_splits=5)
for train_index,test_index in tsSplit.split(X):
    X_train, X_test = X[:len(train_index)], X[len(train_index): (len(train_index)+len(test_index))]
    y_train, y_test = y[:len(train_index)], y[len(train_index): (len(train_index)+len(test_index))]

In [23]:
np.shape(X_train),np.shape(y_train),np.shape(X_test),np.shape(y_test)


((1555, 8, 1), (1555,), (311, 8, 1), (311,))

In [29]:
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense
from keras.utils import to_categorical


In [30]:

def createLSTM():
    model=Sequential()
    model.add(LSTM(200,input_shape=(8,1),return_sequences=True))
    model.add(LSTM(200))
    model.add(Dense(200))
    model.add(Dropout(0.20))
    model.add(Dense(50))
    model.add(Dropout(0.20))
    model.add(Dense(1))
    return model

In [31]:
model=createLSTM()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 8, 200)            161600    
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                10050     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total para

In [32]:
model.compile(loss='mae',optimizer='adam',metrics=['acc'])
model.fit(X_train,y_train,epochs=300,verbose=1)

Epoch 1/300
1555/1555 [==============================] - 4s 3ms/step - loss: 68.8812 - acc: 6.4309e-04
Epoch 2/300
1555/1555 [==============================] - 2s 1ms/step - loss: 20.9546 - acc: 6.4309e-04
Epoch 3/300
1555/1555 [==============================] - 2s 1ms/step - loss: 20.3407 - acc: 6.4309e-04
Epoch 4/300
1555/1555 [==============================] - 2s 1ms/step - loss: 16.0573 - acc: 0.0032
Epoch 5/300
1555/1555 [==============================] - 2s 1ms/step - loss: 15.1618 - acc: 6.4309e-04
Epoch 6/300
1555/1555 [==============================] - 2s 1ms/step - loss: 12.4183 - acc: 0.0013
Epoch 7/300
1555/1555 [==============================] - 2s 1ms/step - loss: 14.2746 - acc: 6.4309e-04
Epoch 8/300
1555/1555 [==============================] - 2s 1ms/step - loss: 13.1059 - acc: 6.4309e-04
Epoch 9/300
1555/1555 [==============================] - 2s 1ms/step - loss: 11.7310 - acc: 0.0013
Epoch 10/300
1555/1555 [==============================] - 2s 1ms/step - loss: 11.4522

1555/1555 [==============================] - 2s 1ms/step - loss: 11.3360 - acc: 0.0026
Epoch 82/300
1555/1555 [==============================] - 2s 1ms/step - loss: 12.3026 - acc: 6.4309e-04
Epoch 83/300
1555/1555 [==============================] - 2s 1ms/step - loss: 11.8552 - acc: 0.0013
Epoch 84/300
1555/1555 [==============================] - 2s 1ms/step - loss: 10.2059 - acc: 0.0032
Epoch 85/300
1555/1555 [==============================] - 2s 1ms/step - loss: 10.0125 - acc: 0.0019
Epoch 86/300
1555/1555 [==============================] - 2s 1ms/step - loss: 11.0839 - acc: 0.0032
Epoch 87/300
1555/1555 [==============================] - 2s 1ms/step - loss: 11.5731 - acc: 0.0019
Epoch 88/300
1555/1555 [==============================] - 2s 1ms/step - loss: 11.1004 - acc: 0.0026
Epoch 89/300
1555/1555 [==============================] - 2s 1ms/step - loss: 11.0821 - acc: 0.0026
Epoch 90/300
1555/1555 [==============================] - 2s 1ms/step - loss: 10.4716 - acc: 0.0013
Epoch 91/

1555/1555 [==============================] - 2s 1ms/step - loss: 9.7537 - acc: 0.0045
Epoch 163/300
1555/1555 [==============================] - 2s 1ms/step - loss: 10.6620 - acc: 0.0019
Epoch 164/300
1555/1555 [==============================] - 2s 1ms/step - loss: 10.3311 - acc: 6.4309e-04
Epoch 165/300
1555/1555 [==============================] - 2s 1ms/step - loss: 11.0870 - acc: 0.0032
Epoch 166/300
1555/1555 [==============================] - 2s 1ms/step - loss: 10.0907 - acc: 0.0026
Epoch 167/300
1555/1555 [==============================] - 2s 1ms/step - loss: 10.5317 - acc: 0.0013
Epoch 168/300
1555/1555 [==============================] - 2s 1ms/step - loss: 10.2095 - acc: 0.0013
Epoch 169/300
1555/1555 [==============================] - 2s 1ms/step - loss: 9.6361 - acc: 0.0026
Epoch 170/300
1555/1555 [==============================] - 2s 1ms/step - loss: 10.0424 - acc: 0.0019
Epoch 171/300
1555/1555 [==============================] - 2s 1ms/step - loss: 9.8194 - acc: 0.0026
Epo

1555/1555 [==============================] - 2s 1ms/step - loss: 9.4735 - acc: 0.0032
Epoch 244/300
1555/1555 [==============================] - 2s 1ms/step - loss: 9.5112 - acc: 0.0026
Epoch 245/300
1555/1555 [==============================] - 2s 1ms/step - loss: 11.7160 - acc: 0.0019
Epoch 246/300
1555/1555 [==============================] - 2s 1ms/step - loss: 11.1762 - acc: 0.0013
Epoch 247/300
1555/1555 [==============================] - 2s 1ms/step - loss: 11.3941 - acc: 6.4309e-04
Epoch 248/300
1555/1555 [==============================] - 2s 1ms/step - loss: 9.9245 - acc: 0.0013
Epoch 249/300
1555/1555 [==============================] - 2s 1ms/step - loss: 10.2599 - acc: 0.0032
Epoch 250/300
1555/1555 [==============================] - 2s 1ms/step - loss: 13.6282 - acc: 0.0013
Epoch 251/300
1555/1555 [==============================] - 2s 1ms/step - loss: 11.0691 - acc: 0.0039
Epoch 252/300
1555/1555 [==============================] - 2s 1ms/step - loss: 10.3028 - acc: 0.0019
Epo

In [33]:
##lets try to test this on the dataset


In [34]:
model.evaluate(X_test,y_test)

311/311 [==============================] - 0s 1ms/step


[16.141260208424264, 0.0]

In [37]:
test=all_close_df[all_close_df['SYMBOL']=='IBM']
idxTest=np.random.randint(len(test))
testVals=test.iloc[idxTest:idxTest+7][['Date','Close']]
testVals['Date']

12140561    1992-12-24
12140562    1992-12-28
12140563    1992-12-29
12140564    1992-12-30
12140565    1992-12-31
12140566    1993-01-04
12140567    1993-01-05
Name: Date, dtype: object

In [38]:
idx=np.random.randint(len(X_test))
test=X_test[idx]
print(test)
test=np.reshape(test,(1,8,1))
preds=model.predict(test)
print('PRED:',preds[0],' ACT:',y_test[idx])

[['20110426']
 ['262.52']
 ['266.41']
 ['268.88']
 ['268.98']
 ['272.05']
 ['269.28']
 ['266.94']]
PRED: [265.1324]  ACT: 267.9


In [40]:
print(len(preds))

1


In [42]:
print(len(X_test))

311


In [43]:
print(len(y_test))

311


In [44]:
preds = model.predict(X_test)

In [45]:
print(len(preds))

311


In [46]:
for i in range(len(preds)):
    print('the prediction is  : ',preds[i] ,' and the actual prediction  is  : ',y_test[i])

the prediction is  :  [175.20459]  and the actual prediction  is  :  172.04
the prediction is  :  [183.13371]  and the actual prediction  is  :  184.89
the prediction is  :  [186.34114]  and the actual prediction  is  :  189.75
the prediction is  :  [193.13046]  and the actual prediction  is  :  194.74
the prediction is  :  [199.81828]  and the actual prediction  is  :  201.5
the prediction is  :  [195.10799]  and the actual prediction  is  :  193.75
the prediction is  :  [196.39653]  and the actual prediction  is  :  202.18
the prediction is  :  [213.43968]  and the actual prediction  is  :  220.14
the prediction is  :  [209.80481]  and the actual prediction  is  :  208.38
the prediction is  :  [201.8181]  and the actual prediction  is  :  204.64
the prediction is  :  [202.88174]  and the actual prediction  is  :  204.8
the prediction is  :  [209.49152]  and the actual prediction  is  :  219.08
the prediction is  :  [217.24217]  and the actual prediction  is  :  223.36
the prediction 

In [47]:
## lets try to save this model


In [48]:
!ls


__notebook_source__.ipynb


In [49]:
!mkdir model

In [50]:
!ls


__notebook_source__.ipynb  model


In [51]:
# serialize model to JSON
model_json = model.to_json()
with open("./model/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("./model/model.h5")
print("Saved model to disk")


Saved model to disk


In [52]:
!ls


__notebook_source__.ipynb  model


In [53]:
!cd model

In [54]:
!ls

__notebook_source__.ipynb  model


In [63]:
!git clone https://github.com/sarvinder/stock_market.git

Cloning into 'stock_market'...
fatal: unable to access 'https://github.com/sarvinder/stock_market.git/': Could not resolve host: github.com
